In [10]:
import torch
import json
from torch.utils.data import Dataset, DataLoader
from transformers import BertTokenizer, BertForSequenceClassification

#Dataset Prep

In [11]:
    !curl https://raw.githubusercontent.com/pytorch/xla/master/contrib/scripts/env-setup.py -o pytorch-xla-env-setup.py
    !python pytorch-xla-env-setup.py --version nightly --apt-packages libomp5 libopenblas-dev

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  6028  100  6028    0     0  40988      0 --:--:-- --:--:-- --:--:-- 41287
Updating... This may take around 2 minutes.
Found existing installation: torch 2.1.0
Uninstalling torch-2.1.0:
  Successfully uninstalled torch-2.1.0
Found existing installation: torchvision 0.16.0
Uninstalling torchvision-0.16.0:
  Successfully uninstalled torchvision-0.16.0
Traceback (most recent call last):
  File "/kaggle/working/pytorch-xla-env-setup.py", line 182, in <module>
    run_setup(args)
  File "/kaggle/working/pytorch-xla-env-setup.py", line 156, in run_setup
    install_vm(version, args.apt_packages, is_root=not args.tpu)
  File "/kaggle/working/pytorch-xla-env-setup.py", line 140, in install_vm
    subprocess.call(cmd)
  File "/usr/local/lib/python3.10/subprocess.py", line 345, in call
    with Popen(*popenargs, **kwargs) as p:
  File "

In [ ]:
import json
import random

with open('/content/train_unsupervised.json', 'r') as f:
    cf_data = [json.loads(line) for line in f]

number_of_records = len(cf_data)
print(number_of_records)

96866


In [ ]:
# Define split sizes for a 75-25 split
cf_subsample = int(0.02 * len(cf_data))
cf_ss_data = cf_data[:cf_subsample]
number_of_records = len(cf_ss_data)
print(number_of_records)

1937


In [ ]:
# Remove 'original_ending' from each dictionary
cf_ss_data_no_ending = [{k: v for k, v in d.items() if k != 'original_ending'} for d in cf_ss_data]

# Print the updated JSON dataset
cf_ss_data_no_ending[0:1]

[{'story_id': 'e6be6391-eff3-4ef3-8799-94df3ae7e3e5',
  'premise': 'Gina and her class had to write speeches.',
  'initial': 'Gina decided she would use dolphins as a topic.',
  'counterfactual': 'But Gina cannot think of any topic to write on.',
  'label': 1}]

In [ ]:
# Define the percentage for label 1
label_1_percentage = 10

# Shuffle the dataset randomly
random.shuffle(cf_ss_data_no_ending)

# Calculate the number of data points for label 1 based on the percentage
label_1_count = int(len(cf_ss_data_no_ending) * (label_1_percentage / 100))

# Assign label 1 to the specified percentage and label 0 to the rest
for i, data_point in enumerate(cf_ss_data_no_ending):
    if i < label_1_count:
        # Assign label 1
        data_point['label'] = 1
    else:
        # Assign label 0
        data_point['label'] = 0

# Shuffle the dataset randomly
#random.shuffle(cf_ss_data)

In [ ]:
cf_ss_data_no_ending[0:5]

[{'story_id': 'dd416a3f-d4ad-4314-b1c4-be55ec26ea99',
  'premise': 'Our dog ran away.',
  'initial': 'We looked everywhere for him.',
  'counterfactual': 'We looked everywhere and found him.',
  'label': 1},
 {'story_id': '73010332-6da0-4e4b-a7b2-c6d03c25a429',
  'premise': 'May really wanted to go to the concert that was playing next week.',
  'initial': 'She tried to buy tickets online, but they were sold out.',
  'counterfactual': 'May was able to buy tickets online.',
  'label': 1},
 {'story_id': '2872848e-c64d-4fcf-a741-3a1ed2aa2d00',
  'premise': 'I had a big paper due in my writing class.',
  'initial': 'I had made a mistake with my pencil and tried to erase it.',
  'counterfactual': "I had made a mistake with my pen and couldn't erase it.",
  'label': 1},
 {'story_id': '4bc3ea62-bf07-428e-b763-f438679b42af',
  'premise': 'James liked to buy scratch tickets on payday for fun.',
  'initial': 'One day he won a hundred dollars.',
  'counterfactual': 'One day he lost five dollars.',

In [ ]:
# Initialize a dictionary to store the count of each 'label' value
label_counts = {}

# Count the occurrences of each 'label' value
for record in cf_ss_data:
    label = record.get('label')  # Assuming 'label' is the key in each record
    if label in label_counts:
        label_counts[label] += 1
    else:
        label_counts[label] = 1

# Display the count of each 'label' value
for label, count in label_counts.items():
    print(f"Label: {label}, Count: {count}")

Label: 1, Count: 193

Label: 0, Count: 1744


In [ ]:
#Spilt the dataset 1937 sub-sampled records into train and test
import json
import random

# Define the ratio of records to be in the first dataset (e.g., 70%)
train_dataset_ratio = 0.7

# Randomly assign each record to one of the two datasets
train_dataset_final = [record for record in cf_ss_data_no_ending if random.random() < train_dataset_ratio]
test_dataset_final = [record for record in cf_ss_data_no_ending if record not in train_dataset_final]

# Display the sizes of the two datasets
print(f"Size of the train dataset: {len(train_dataset_final)}")
print(f"Size of the test dataset: {len(test_dataset_final)}")

# Write train dataset to JSON file
with open('train_dataset_final.json', 'w') as train_file:
    json.dump(train_dataset_final, train_file)

# Write test dataset to JSON file
with open('test_dataset_final.json', 'w') as test_file:
    json.dump(test_dataset_final, test_file)

# Save the two datasets to separate JSON files
#with open('train_dataset_final.json', 'w') as first_file:
   # json.dump(train_dataset_final, first_file, indent=2)

#with open('test_dataset_final.json', 'w') as second_file:
    #json.dump(test_dataset_final, second_file, indent=2)


Size of the train dataset: 1338

Size of the test dataset: 599


#Model Training

In [12]:
# Start training from this cell onward - Reading the train dataset back across models
with open('/kaggle/input/cf-dataset/train_dataset_final.json', 'r') as train_file:
    train_dataset_back = json.load(train_file)

# Reading the test dataset back
with open('/kaggle/input/cf-dataset/test_dataset_final.json', 'r') as test_file:
    test_dataset_back = json.load(test_file)

# Display the sizes of the two datasets
print(f"Size of the train dataset: {len(train_dataset_back)}")
print(f"Size of the test dataset: {len(test_dataset_back)}")

Size of the train dataset: 1338
Size of the test dataset: 599


In [13]:
train_dataset_back[0:3]

[{'story_id': '5ef7fdfa-881c-4b08-ba22-18d3b6289676',
  'premise': 'Jim worked at an ice cream factory as the foreman.',
  'initial': 'One day the vats of ice cream began running over.',
  'counterfactual': 'One day the vats of ice cream were becoming less.',
  'label': 1},
 {'story_id': 'e5955040-5b87-4acb-a8c7-7e81d0ffb9f5',
  'premise': 'Susie was sitting on her barstool.',
  'initial': 'She kept kicking the counter with her feet.',
  'counterfactual': 'She kept herself steady with her feet.',
  'label': 1},
 {'story_id': 'd8d04710-4d4e-4bcb-a9e2-ebf99c394660',
  'premise': 'During the Holiday Season my neighbor, Mary, feels lonely.',
  'initial': 'Her husband died a few years ago and her son moved away last summer.',
  'counterfactual': 'Mary decided to take a cruise for the holidays.',
  'label': 1}]

In [14]:
from torch.utils.data import Dataset

class TTDataset(Dataset):
    def __init__(self, copa_data, tokenizer):
        self.copa_data = copa_data
        self.tokenizer = tokenizer

    def __len__(self):
        return len(self.copa_data)

    def __getitem__(self, idx):
        data = self.copa_data[idx]
        premise = data['premise']
        hypotheses = [data['initial'], data['counterfactual']]
        label = data['label']

        # Tokenize
        encoded_input = self.tokenizer([premise] * 2, hypotheses, padding='max_length', truncation=True, return_tensors='pt')
        encoded_input['labels'] = torch.tensor(label)

        return encoded_input


In [15]:
from transformers import BertTokenizer, BertForMultipleChoice
import torch
from torch.utils.data import DataLoader
import torch.optim as optim

# Load tokenizer and model
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertForMultipleChoice.from_pretrained('bert-base-uncased')


# Prepare the DataLoader
TT_Dataset = TTDataset(train_dataset_back, tokenizer)
dataloader = DataLoader(TT_Dataset, batch_size=16, shuffle=True)

Some weights of BertForMultipleChoice were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [18]:
from transformers import BertTokenizer, BertForMultipleChoice
import torch
from torch.utils.data import DataLoader
import torch.optim as optim

# Optimizer and Loss Function
optimizer = optim.Adam(model.parameters(), lr=1e-5)
loss_fn = torch.nn.CrossEntropyLoss()

# Training Loop
model.train()
for epoch in range(3):  # Number of epochs
    for batch in dataloader:
        optimizer.zero_grad()

        input_ids = batch['input_ids'].squeeze(1)
        attention_mask = batch['attention_mask'].squeeze(1)
        labels = batch['labels']

        outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss

        loss.backward()
        optimizer.step()

        print(f"Epoch: {epoch}, Loss: {loss.item()}")


Epoch: 0, Loss: 0.6799638867378235
Epoch: 0, Loss: 0.691728949546814
Epoch: 0, Loss: 0.6899026036262512
Epoch: 0, Loss: 0.7013248801231384
Epoch: 0, Loss: 0.6778404712677002
Epoch: 0, Loss: 0.6976286768913269
Epoch: 0, Loss: 0.6868414878845215
Epoch: 0, Loss: 0.7264204025268555
Epoch: 0, Loss: 0.7360163927078247
Epoch: 0, Loss: 0.6992794275283813
Epoch: 0, Loss: 0.7001288533210754
Epoch: 0, Loss: 0.6891927719116211
Epoch: 0, Loss: 0.6802003383636475
Epoch: 0, Loss: 0.7023868560791016
Epoch: 0, Loss: 0.7215389013290405
Epoch: 0, Loss: 0.7314345240592957
Epoch: 0, Loss: 0.6971152424812317
Epoch: 0, Loss: 0.7033888697624207
Epoch: 0, Loss: 0.6980613470077515
Epoch: 0, Loss: 0.6963198781013489
Epoch: 0, Loss: 0.685186505317688
Epoch: 0, Loss: 0.6818411350250244
Epoch: 0, Loss: 0.6834957599639893
Epoch: 0, Loss: 0.7236918807029724
Epoch: 0, Loss: 0.7317310571670532
Epoch: 0, Loss: 0.6482637524604797
Epoch: 0, Loss: 0.6863251328468323
Epoch: 0, Loss: 0.7007505893707275
Epoch: 0, Loss: 0.6873

In [19]:
validation_dataset = TTDataset(test_dataset_back, tokenizer)
validation_dataloader = DataLoader(validation_dataset, batch_size=8)


In [20]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

model.eval()
correct_predictions = 0
total_predictions = 0

accuracy, f1, precision, recall = [], [], [], []
batches = 0

def compute_metrics(y_true, y_pred):
    accuracy = accuracy_score(y_true, y_pred)
    f1 = f1_score(y_true, y_pred)
    precision = precision_score(y_true, y_pred)
    recall = recall_score(y_true, y_pred)
    return [accuracy, f1, precision, recall]

with torch.no_grad():
    for batch in validation_dataloader:
        input_ids = batch['input_ids'].squeeze(1)
        attention_mask = batch['attention_mask'].squeeze(1)
        labels = batch['labels']  # ground truth

        outputs = model(input_ids=input_ids, attention_mask=attention_mask)
        predictions = torch.argmax(outputs.logits, dim=1)  # predictions

        calculated_metrics = compute_metrics(labels, predictions)
        accuracy.append(calculated_metrics[0])
        f1.append(calculated_metrics[1])
        precision.append(calculated_metrics[2])
        recall.append(calculated_metrics[3])
        batches += 1

        correct_predictions += (predictions == labels).sum().item()
        total_predictions += labels.size(0)

overall_accuracy = correct_predictions / total_predictions
print(f"Overall Accuracy: {overall_accuracy:.4f}")

print(f"Accuracy: {sum(accuracy)/batches:.4f}")
print(f"F1: {sum(f1)/batches:.4f}")
print(f"Precision: {sum(precision)/batches:.4f}")
print(f"Recall: {sum(recall)/batches:.4f}")

print(f'Number of batches: {batches}')

/usr/local/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1497: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1497: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 due to no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1497: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 due to no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1497: Undefine

Overall Accuracy: 0.6811
Accuracy: 0.6814
F1: 0.0417
Precision: 0.1022
Recall: 0.0283
Number of batches: 75


/usr/local/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1497: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 due to no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [5]:
model.eval()
correct_predictions = 0
total_predictions = 0

with torch.no_grad():
    for batch in validation_dataloader:
        input_ids = batch['input_ids'].squeeze(1)
        attention_mask = batch['attention_mask'].squeeze(1)
        labels = batch['labels']

        outputs = model(input_ids=input_ids, attention_mask=attention_mask)
        predictions = torch.argmax(outputs.logits, dim=1)
        correct_predictions += (predictions == labels).sum().item()
        total_predictions += labels.size(0)

overall_accuracy = correct_predictions / total_predictions
print(f"Overall Accuracy: {overall_accuracy:.4f}")


NameError: name 'model' is not defined

In [ ]:
# Specify the directory where you want to save the model
save_directory = '/content'

# Save the model to the specified directory
model.save_pretrained(save_directory)

# Optionally, save the tokenizer as well
model.config.save_pretrained(save_directory)

In [ ]:
from transformers import BertForMultipleChoice

# Specify the directory where the model is saved
load_directory = 'path/to/save/model'

# Load the model from the specified directory
loaded_model = BertForMultipleChoice.from_pretrained(load_directory)


# New Section